In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [128]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pandas as pd
import keras_tuner as kt
import tensorflow_datasets as tfds
import time

import sys
import os
py_file_location = "/content/drive/My Drive"
sys.path.append(os.path.abspath('/content/drive/MyDrive/NeuralNetwork'))
from models_hs import *
from trainer import *
from preprocess import *
from callbacks import *


# 0 Data

Read in data for each dimension and store as `pd.DataFrame`:

In [129]:
def get_dfs(dims, num_samples):
  dataframes_list = []
  for dim in dims:
      temp_df = pd.read_csv(f"/content/drive/MyDrive/data/{num_samples}_basket_data_{dim}.csv")
      temp_df = temp_df.drop(['Unnamed: 0'], axis=1)

      # move column with contract prices to the end
      cols = list(temp_df.columns.values) 
      cols.pop(cols.index('contract_price')) 
      temp_df = temp_df[cols+['contract_price']] 

      dataframes_list.append(temp_df)
  return dataframes_list

In [130]:
dims = [1, 4, 7, 10, 13, 16]
dataframes_list = get_dfs(dims, 1000)
dataframes_list[0]

,days_to_maturity,strike,volatility,mean_volatility,reversion,vol_of_var,rate,Underlying_0,Rho_0,contract_price
0,52.0,16.50,0.010564,0.041048,1.775966,0.058849,0.015002,17.332500,-0.333905,10.938264
1,21.0,49.50,0.039117,0.049353,1.540456,0.489135,0.001440,9.260000,-0.129997,0.571392
2,31.0,14.00,0.029249,0.006491,0.982883,0.224651,0.017221,16.414301,-0.462886,8.445650
3,18.0,8.70,0.038737,0.016305,2.372708,0.329505,0.006641,15.892000,-0.141496,9.129831
4,59.0,16.50,0.043640,0.038880,4.562963,0.706960,0.053901,11.277000,-0.379364,10.687387
...,...,...,...,...,...,...,...,...,...,...
995,38.0,13.00,0.001502,0.046531,1.663993,0.145838,0.001362,12.819120,-0.307483,6.298129
996,85.0,15.00,0.042181,0.044124,0.302489,0.661890,0.018707,18.106250,-0.050286,24.277845
997,29.0,0.48,0.033398,0.005511,4.443828,0.506015,0.002109,67.483701,-0.693189,65.758164
998,7.0,9.50,0.004519,0.036562,2.277263,0.288585,0.003034,15.919620,-0.470891,7.345720


Test model on 10000 samples with dim = 1:

In [132]:
dataframes_list_1 = get_dfs([1], 10000)
dataframes_list_1[0]

,days_to_maturity,strike,volatility,mean_volatility,reversion,vol_of_var,rate,Underlying_0,Rho_0,contract_price
0,24.0,4.250,0.002046,0.050490,0.842923,0.631400,0.022602,58.010400,-0.320407,61.992674
1,119.0,5.050,0.050203,0.048482,2.676144,0.702256,0.001616,39.998899,-0.102169,16.377944
2,10.0,15.000,0.012146,0.009763,0.952558,0.693369,0.009901,5.080200,-0.394712,0.026545
3,235.0,14.000,0.028765,0.038385,4.946453,0.012569,0.001688,13.283750,-0.471352,0.000000
4,130.0,16.300,0.010112,0.045151,0.474549,0.218541,0.015604,4.440500,-0.101970,4.166032
...,...,...,...,...,...,...,...,...,...,...
9995,22.0,3.975,0.003293,0.047075,0.377678,0.601305,0.026390,12.496370,-0.421565,11.578677
9996,7.0,14.000,0.022504,0.025393,0.826864,0.429884,0.001874,15.524250,-0.229666,3.513765
9997,10.0,12.400,0.002174,0.040227,0.190017,0.502809,0.002174,4.612700,-0.243227,0.296504
9998,192.0,4.450,0.044043,0.030118,4.034105,0.391629,0.025649,19.868500,-0.190983,2.275450


# 1 Training

Define some parameters:

In [ ]:
print_num_epochs = 5 # print progress every print_num_epochs epochs

path_to_save = os.path.join(parent_path, 'NeuralNetwork/models/')  # path to save the model

patience = 10  

num_epochs = 10  

output_shape = (1, )

In [140]:
import argparse
from pyexpat import model
import numpy as np
import tensorflow as tf

def model_builder_hs(dim,
            num_layers   = 2,
            hidden_units = [14,7],
            output_shape = (1,),
            activation = 'elu',
            regularizer = None,
            initializer = tf.keras.initializers.he_uniform(),
            final_activation = None,
            dropout = None,
            batchnorm = False
            ): 
    """
    Returns a model for training and testing.  

    Args:
        - dim: int, basket size
        - num_layers: int, number of hidden layers
        - hidden_units: list of number of hidden units in each layer
        - output_shape: shape of the output data
        - activation: string, activation function
        - initializer: initializer for the weights
        - final_activation: string, activation function of final layer
        - dropout: list, dropout rate for each layer, default None
        - batchnorm: bool, specifies if batch normalization is used, default False 
    
    Output:  
        - model: tf.keras.Model, compiled if compile is True
    """  
    assert num_layers == len(hidden_units), "Number of hidden units must match number of layers"
    if dropout is not None:  
        assert num_layers == len(dropout), "Number of dropout rates must match number of layers"

    input_shape = (7 + 2*dim,)

    inputs = tf.keras.layers.Input(shape=input_shape)
    h = tf.keras.layers.Flatten()(inputs)

    for i, layer in enumerate(hidden_units):
        h = tf.keras.layers.Dense(layer, activation=activation, kernel_regularizer= regularizer,
                                  kernel_initializer = initializer)(h)
        if dropout:
            h = tf.keras.layers.Dropout(dropout[i])(h)
        if batchnorm:
            h = tf.keras.layers.BatchNormalization()(h)
    if final_activation is not None:
        outputs = tf.keras.layers.Dense(output_shape[0], activation=final_activation,
                                        kernel_initializer = initializer)(h)
    else:
        outputs = tf.keras.layers.Dense(output_shape[0], 
                                        kernel_initializer = initializer)(h)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)  

    return model   


def tuned_model_hs(hp):
    """
    Returns a compiled hyperModel for keras tuner. 

    """  

    # defining a set of hyperparameters for tuning and a range of values for each
    num_layers = hp.Int('num_layers', min_value=1, max_value=5) 
    activation = hp.Choice('activation', ['elu','tanh', 'relu', 'sigmoid'])
    learning_rate = hp.Float('learning_rate', min_value=10**(-3), max_value=0.01)
    rate_decay = hp.Float('rate_decay', min_value=0.85, max_value=0.9995)
    l1_reg = hp.Float('l1_regularizer', min_value=10**(-8), max_value=10**(-6.5))
    l2_reg = hp.Float('l1_regularizer', min_value=10**(-8), max_value=10**(-6.5))
    batchnorm = hp.Boolean(name = 'batchnorm')
    
    hidden_units, dropouts = [],[]
    for i in range(num_layers):
        hidden_unit = hp.Int(f'units_{i+1}', min_value=5, max_value=7)
        hidden_units.append(hidden_unit)
        dropout = hp.Float(f'dropout_{i+1}', min_value=0.0, max_value=0.5, step=0.1)
        dropouts.append(dropout)

    model = model_builder_hs(dim,
                    num_layers = num_layers, 
                    hidden_units = hidden_units,
                    dropout = dropouts,
                    activation = activation,
                    batchnorm = batchnorm,
                    regularizer = tf.keras.regularizers.l1_l2(l1_reg,l2_reg)
                    )

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        learning_rate, decay_steps = 4000, decay_rate = rate_decay, staircase = True)
    
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule), loss = tf.keras.losses.MeanAbsolutePercentageError(), 
                  metrics = [tf.keras.metrics.MeanSquaredError()])

    return model  

In [135]:
best_models_history = []
best_hp_lists = []

## 1.1 Using Random Tuner:

In [136]:
dims = [1]
for i, dim in enumerate(dims):
    train_ds, valid_ds, test_ds = pipeline1(dataframes_list_1[i].to_numpy(), scaling=False)

    train_copy, valid_copy, test_copy = pipeline1(dataframes_list_1[i].to_numpy(), prefetch=False)


    print(f"Start training for basket option with size {dim}:")

    random_tuner = kt.RandomSearch(
    hypermodel=tuned_model_hs, # the hypermodel to tune # can be tuneLR or tuneLayer
    objective="val_loss", # the objective to optimize
    max_trials=3, # the maximum number of trials to run
    executions_per_trial=2, # the number of models generated on each trial
    overwrite=True, # whether to overwrite previous trials
    directory="hyperparams/RandomSearch", # the directory to save the trials
    project_name=f"basket_option_{dim}", # the name of the project
    )  

    random_tuner.search(train_ds, epochs = 30, validation_data = valid_ds)    

    best_hp_lists.append(random_tuner.get_best_hyperparameters(1)[0])
    models = random_tuner.get_best_models(num_models=1)
    best_model = models[0]

    random_tuner.results_summary()
    
    begin_train = time.time()
    history = best_model.fit(train_ds, epochs = 100, validation_data = valid_ds)
    end_train = time.time()


Trial 3 Complete [00h 00m 49s]
val_loss: 472.1968536376953

Best val_loss So Far: 472.1968536376953
Total elapsed time: 00h 03m 02s
Results summary
Results in hyperparams/RandomSearch/basket_option_1
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 5
activation: relu
learning_rate: 0.0044667064282456455
rate_decay: 0.9699629512861958
l1_regularizer: 1.6693414021157338e-07
batchnorm: False
units_1: 6
dropout_1: 0.1
units_2: 5
dropout_2: 0.4
units_3: 5
dropout_3: 0.30000000000000004
units_4: 6
dropout_4: 0.5
units_5: 5
dropout_5: 0.0
Score: 472.1968536376953
Trial summary
Hyperparameters:
num_layers: 2
activation: tanh
learning_rate: 0.0013295413522680783
rate_decay: 0.8519401867952281
l1_regularizer: 2.0049884334546466e-08
batchnorm: True
units_1: 6
dropout_1: 0.4
units_2: 6
dropout_2: 0.1
units_3: 5
dropout_3: 0.2
units_4: 7
dropout_4: 0.4
Score: 23485.3369140625
Trial summary
Hyperparameters:
num_layers: 4
activation: relu
learning_rate: 0.007088574583965032
rate_deca

In [139]:
print(f"Training time: {end_train - begin_train} seconds")

Training time: 70.80026173591614 seconds


## 1.2 Using Hyperband Tuner:


In [142]:
dims = [1]
for i, dim in enumerate(dims):
    train_ds, valid_ds, test_ds = pipeline1(dataframes_list_1[i].to_numpy(), scaling=False)

    train_copy, valid_copy, test_copy = pipeline1(dataframes_list_1[i].to_numpy(), prefetch=False)


    print(f"Start training for basket option with size {dim}:")

    Hyperband_tuner = kt.Hyperband(
    hypermodel=tuned_model_hs, # the hypermodel to tune # can be tuneLR or tuneLayer
    objective="val_loss", # the objective to optimize
    max_epochs=10,
    factor=3,
    directory="hyperparams/HyperbandSearch", # the directory to save the trials
    project_name=f"basket_option_{dim}", # the name of the project
    )  

    Hyperband_tuner.search(train_ds, epochs = 30, validation_data = valid_ds)    

    best_hp_lists.append(Hyperband_tuner.get_best_hyperparameters(1)[0])
    models = Hyperband_tuner.get_best_models(num_models=1)
    best_model = models[0]

    Hyperband_tuner.results_summary()
    
    begin_train = time.time()
    history = best_model.fit(train_ds, epochs = 30, validation_data = valid_ds)
    end_train = time.time()


Trial 30 Complete [00h 00m 12s]
val_loss: 6234.1611328125

Best val_loss So Far: 600.0084228515625
Total elapsed time: 00h 02m 41s
Results summary
Results in hyperparams/HyperbandSearch/basket_option_1
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 5
activation: relu
learning_rate: 0.0011543003071872165
rate_decay: 0.9571126414036437
l1_regularizer: 2.613237022535335e-07
batchnorm: False
units_1: 5
dropout_1: 0.5
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
units_2: 5
dropout_2: 0.0
units_3: 5
dropout_3: 0.0
units_4: 5
dropout_4: 0.0
units_5: 5
dropout_5: 0.0
Score: 600.0084228515625
Trial summary
Hyperparameters:
num_layers: 5
activation: elu
learning_rate: 0.005835978452452756
rate_decay: 0.8652186237061492
l1_regularizer: 7.989614041811944e-08
batchnorm: False
units_1: 6
dropout_1: 0.0
units_2: 6
dropout_2: 0.4
units_3: 6
dropout_3: 0.0
units_4: 7
dropout_4: 0.0
units_5: 7
dropout_5: 0.30000000000000004
tuner/epochs: 10
tuner/initial_epoc

In [143]:
print(f"Training time: {end_train - begin_train} seconds")

Training time: 26.55580735206604 seconds


## 1.3 Using Bayesian Optimization

In [145]:
dims = [1]
for i, dim in enumerate(dims):
    train_ds, valid_ds, test_ds = pipeline1(dataframes_list_1[i].to_numpy(), scaling=False)

    train_copy, valid_copy, test_copy = pipeline1(dataframes_list_1[i].to_numpy(), prefetch=False)


    print(f"Start training for basket option with size {dim}:")

    bayes_tuner = kt.BayesianOptimization(
    hypermodel=tuned_model_hs, # the hypermodel to tune # can be tuneLR or tuneLayer
    objective="val_loss", # the objective to optimize
    max_trials=3,
    directory="hyperparams/BayesianOptimization", # the directory to save the trials
    project_name=f"basket_option_{dim}", # the name of the project
    )  

    bayes_tuner.search(train_ds, epochs = 30, validation_data = valid_ds)    

    best_hp_lists.append(bayes_tuner.get_best_hyperparameters(1)[0])
    models = bayes_tuner.get_best_models(num_models=1)
    best_model = models[0]

    bayes_tuner.results_summary()
    
    begin_train = time.time()
    history = best_model.fit(train_ds, epochs = 30, validation_data = valid_ds)
    end_train = time.time()


Trial 3 Complete [00h 00m 36s]
val_loss: 114491.828125

Best val_loss So Far: 13906.3056640625
Total elapsed time: 00h 01m 36s
Results summary
Results in hyperparams/BayesianOptimization/basket_option_1
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
activation: sigmoid
learning_rate: 0.008117221165611607
rate_decay: 0.8955589475710379
l1_regularizer: 2.1512359334152384e-07
batchnorm: True
units_1: 6
dropout_1: 0.2
units_2: 5
dropout_2: 0.0
Score: 13906.3056640625
Trial summary
Hyperparameters:
num_layers: 1
activation: sigmoid
learning_rate: 0.008447677757371937
rate_decay: 0.9058101961212783
l1_regularizer: 6.642022198638799e-08
batchnorm: False
units_1: 6
dropout_1: 0.30000000000000004
Score: 16483.38671875
Trial summary
Hyperparameters:
num_layers: 5
activation: sigmoid
learning_rate: 0.0064758658584953025
rate_decay: 0.85
l1_regularizer: 3.162277660168379e-07
batchnorm: True
units_1: 6
dropout_1: 0.0
units_2: 5
dropout_2: 0.0
units_3: 5
dropout_3: 0.0
units_4: 

In [146]:
print(f"Training time: {end_train - begin_train} seconds")

Training time: 34.867894411087036 seconds
